In [ ]:
import requests
from bs4 import BeautifulSoup
def timesofindia():
    url = "https://timesofindia.indiatimes.com/home/headlines"
    page_request = requests.get(url)
    data = page_request.content
    soup = BeautifulSoup(data,"html.parser")

    counter = 0
    for divtag in soup.find_all('div', {'class': 'headlines-list'}):
        for ultag in divtag.find_all('ul', {'class': 'clearfix'}):
            if (counter <= 10):
                for litag in ultag.find_all('li'):
                    counter = counter + 1
                    print(str(counter) + " - https://timesofindia.indiatimes.com" + litag.find('a')['href'])
                    #print(str(counter) + "." + litag.text + " - https://timesofindia.indiatimes.com" + litag.find('a')['href'])

if __name__ == "__main__":
    timesofindia()

In [ ]:
Hindu business line | Markets | Simple Sentiment

In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    # Classify the sentiment as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity == 0:
        return "Neutral"
    else:
        return "Negative"

def read_html_data(html_content):
    try:
        soup = BeautifulSoup(html_content, "html.parser")

        for li_tag in soup.find_all('li', {'class': 'timeline-item'}):
            article_body_span = li_tag.find('span', {'itemprop': 'articleBody'})
            if article_body_span:
                for ptag in article_body_span.find_all('p'):
                    sentiment = get_sentiment(ptag.get_text(strip=True))
                    print(f"Paragraph Content: {ptag.get_text(strip=True)}")
                    print(f"Sentiment: {sentiment}\n")
    except Exception as e:
        print(f"Error parsing the HTML: {e}")

def economictimes_headlines():
    print("Fetching headlines from Economic Times...")
        #url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-02-january-2024/article67695347.ece"
        url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-03-january-2024/article67698291.ece"

    try:
        page_request = requests.get(url)
        page_request.raise_for_status()  # Raise an exception for bad status codes
        print("Status:", page_request.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error making the request: {e}")
        return

    try:
        data = page_request.content
        read_html_data(data)
    except Exception as e:
        print(f"Error parsing the page: {e}")

if __name__ == "__main__":
    economictimes_headlines()


In [ ]:
Sentiment Analyze All content | Hindu Business Line

In [20]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def analyze_sentiment(text):
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    
    try:
        result = sentiment_analysis(text)
        if result:
            return result[0]
        else:
            return None
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return None

def read_html_data(html_content):
    try:
        soup = BeautifulSoup(html_content, "html.parser")

        for li_tag in soup.find_all('li', {'class': 'timeline-item'}):
            article_body_span = li_tag.find('span', {'itemprop': 'articleBody'})
            if article_body_span:
                collected_content = []  # Collect all content from <p> tags
                for ptag in article_body_span.find_all('p'):
                    content = ptag.get_text(strip=True)
                    collected_content.append(content)
                
                # Check if collected_content is not empty before analyzing sentiment
                if collected_content:
                    sentiment_result = analyze_sentiment(collected_content)
                    print(f"Paragraph Content: {collected_content}")
                    print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.4f})\n")

    except Exception as e:
        print(f"Error parsing the HTML: {e}")

def economictimes_headlines():
    print("Fetching headlines from Economic Times...")
    #url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-02-january-2024/article67695347.ece"
    url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-03-january-2024/article67698291.ece"
    
    try:
        page_request = requests.get(url)
        page_request.raise_for_status()
        print("Status:", page_request.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error making the request: {e}")
        return

    try:
        data = page_request.content
        read_html_data(data)
    except Exception as e:
        print(f"Error parsing the page: {e}")

if __name__ == "__main__":
    economictimes_headlines()


Fetching headlines from Economic Times...
Status: 200
Paragraph Content: ['Pidilite Industrieshas received order from the Assistant Commissioner of State Tax, Royapettah, Chennai, Tamil Nadu imposing penalty of ₹6,57,640 under applicable provisions of the Central Goods and Service Tax Act, 2017. GST department has levied penalty of for the disallowance of ITC availed as the same is not appearing in GSTR-2A.', 'The Company reasonably expects a favourable outcome at the appellate level, it said in a regulatory filing.', 'The stock trades at ₹2,730.55 on the NSE, down by 0.26%.']
Sentiment: 1 star (confidence: 0.6373)

Paragraph Content: ['NSE niftywasdown by 0.39 per cent or 85.10 to 21,578.70 points, while theBSE sensexwas at 71,548 down by 0.47 per cent or 337.11 points.']
Sentiment: 1 star (confidence: 0.5207)

Paragraph Content: ['Nifty 50 continues to trade lower. It is currently trading at 21,588, down 0.36 per cent. Read morehere.']
Sentiment: 1 star (confidence: 0.4320)

Paragrap

In [ ]:
Read the scrip code from file and analyse only those | Last Good working

In [46]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

def load_sentiment_keywords():
    # Load the keywords from the Excel sheet
    excel_path = r"C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx"
    sheet_name = "Sentiment"
    
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        return set(df['Keyword'].str.lower())  # Convert to set for efficient lookup
    except Exception as e:
        print(f"Error loading sentiment keywords: {e}")
        return set()


def analyze_sentiment(text):
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    try:
        result = sentiment_analysis(text)
        if result:
            return result[0]
        else:
            return None
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return None

# def read_html_data(html_content, sentiment_keywords):
#     try:
#         soup = BeautifulSoup(html_content, "html.parser")

#         for li_tag in soup.find_all('li', {'class': 'timeline-item'}):
#             article_body_span = li_tag.find('span', {'itemprop': 'articleBody'})
#             if article_body_span:
#                 collected_content = []  # Collect all content from <p> tags
#                 for ptag in article_body_span.find_all('p'):
#                     content = ptag.get_text(strip=True)
#                     collected_content.append(content)

#                 # Combine conditions for analyzing sentiment and checking sentiment keywords
#                 if collected_content and any(keyword in ' '.join(collected_content).lower() for keyword in sentiment_keywords):
#                     sentiment_result = analyze_sentiment(' '.join(collected_content))
#                     if sentiment_result:
#                         print(f"Paragraph Content: {' '.join(collected_content)}")
#                         print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.4f})\n")
#                     else:
#                         print("Sentiment analysis failed.")
#                 # else:
#                 #     # Log that the sentiment analysis was skipped
#                 #     print("Sentiment analysis skipped. No content or no sentiment keywords found.")

#     except Exception as e:
#         print(f"Error parsing the HTML: {e}")

def read_html_data(html_content, sentiment_keywords):
    try:
        soup = BeautifulSoup(html_content, "html.parser")

        for li_tag in soup.find_all('li', {'class': 'timeline-item'}):
            article_body_span = li_tag.find('span', {'itemprop': 'articleBody'})
            headline_h3 = li_tag.find('h3', {'itemprop': 'headline'})

            # Use collected_content for both span and h3 tags
            collected_content = []

            if article_body_span and not article_body_span.contents:
                # If span tag is empty, use content from h3 tag
                content = headline_h3.get_text(strip=True)
                collected_content.append(content)
            elif article_body_span:
                # Collect content from the span tag
                collected_content.extend(ptag.get_text(strip=True) for ptag in article_body_span.find_all('p'))
            else:
                continue  # Move to the next iteration if both span and h3 are not found

            print(f"Paragraph Content: {' '.join(collected_content)}")

            # Continue with sentiment analysis logic
            if collected_content and any(keyword in ' '.join(collected_content).lower() for keyword in sentiment_keywords):
                sentiment_result = analyze_sentiment(' '.join(collected_content))
                if sentiment_result:
                    print(f"Paragraph Content: {' '.join(collected_content)}")
                    print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.4f})\n")
                else:
                    print("Sentiment analysis failed.")

    except Exception as e:
        print(f"Error parsing the HTML: {e}")



def economictimes_headlines():
    print("Fetching headlines from Economic Times...")
    #url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-02-january-2024/article67695347.ece"
    #url = "https://www.thehindubusinessline.com/markets/share-market-nifty-sensex-live-updates-03-january-2024/article67698291.ece"
    url = "https://www.thehindubusinessline.com/markets/stock-markets/share-market-nifty-sensex-live-updates-04-january-2024/article67702215.ece"

    
    try:
        page_request = requests.get(url)
        page_request.raise_for_status()
        print("Status:", page_request.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error making the request: {e}")
        return

    try:
        data = page_request.content
        sentiment_keywords = load_sentiment_keywords()
        read_html_data(data, sentiment_keywords)
    except Exception as e:
        print(f"Error parsing the page: {e}")

if __name__ == "__main__":
    economictimes_headlines()


Fetching headlines from Economic Times...
Status: 200
Paragraph Content: Bharat Aluminium Company Limited, a subsidiary of Vedanta Limited, has received an Order from The Office of the Commissioner, Raipur, confirming demand of GST pertaining to FY 2017-18 on account of procedural issue related to transfer of credit of erstwhile taxes to GST regime in terms of relevant provisions of the Central Goods and Services Tax Act, 2017 read with Integrated Goods and Services Tax Act, 2017 (‘GST Law’). BALCO evaluated the merits of the case and decided to file the appeal with the Appellate Authority within the time limit prescribed under the GST Law. Vedanta stock traded at ₹265.45 on the NSE, up by 2.83%.
Paragraph Content: Zydus Wellnessstock trades at ₹1,689.50 on the NSE, up by 0.94%. The company is in receipt of Order under CGST Act, 2017.
Paragraph Content: GlaxoSmithKline Pharmaceuticalsinformed the exchanges that the Delhi High Court ruled in favour of the company. It has quashed and set

In [ ]:
Ecomonic times | Live Blog

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

def load_sentiment_keywords():
    # Load the keywords from the Excel sheet
    excel_path = r"C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx"
    sheet_name = "Sentiment"
    
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        return set(df['Keyword'].str.lower())  # Convert to set for efficient lookup
    except Exception as e:
        print(f"Error loading sentiment keywords: {e}")
        return set()

def analyze_sentiment(text):
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    result = sentiment_analysis(text)
    return result[0]

def read_html_data(html_content, sentiment_keywords):
    try:
        soup = BeautifulSoup(html_content, "html.parser")

        for h2_tag in soup.find_all('h2', {'class': ''}):
            title = h2_tag.get_text(strip=True)
            print(f"Title: {title}")

            div_tag = h2_tag.find_next('div', {'class': 'blogSysn'})
            if div_tag:
                content = div_tag.get_text(strip=True)
                print(f"Paragraph Content: {content}")
                # Check if any sentiment keywords are present in the content
                # if any(keyword in content.lower() for keyword in sentiment_keywords):
                #     sentiment_result = analyze_sentiment(content)
                #     print(f"Paragraph Content: {content}")
                #     print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.4f})\n")

    except Exception as e:
        print(f"Error parsing the HTML: {e}")

def economictimes_headlines():
    print("Fetching headlines from Economic Times...")
    url = "https://economictimes.indiatimes.com/markets/stocks/live-blog/bse-sensex-today-live-nifty-stock-market-updates-02-january-2024/liveblog/msid-106462685,curpg-2.cms"

    try:
        page_request = requests.get(url)
        page_request.raise_for_status()
        print("Status:", page_request.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error making the request: {e}")
        return

    try:
        data = page_request.content
        sentiment_keywords = load_sentiment_keywords()
        read_html_data(data, sentiment_keywords)
    except Exception as e:
        print(f"Error parsing the page: {e}")

if __name__ == "__main__":
    economictimes_headlines()



In [ ]:
StockEdge scraping /// Parked not working

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

def load_sentiment_keywords():
    # Load the keywords from the Excel sheet
    excel_path = r"C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx"
    sheet_name = "Sentiment"
    
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        return set(df['Keyword'].str.lower())  # Convert to set for efficient lookup
    except Exception as e:
        print(f"Error loading sentiment keywords: {e}")
        return set()

def analyze_sentiment(text):
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    result = sentiment_analysis(text)
    return result[0]

def read_html_data(html_content, sentiment_keywords):
    try:
        soup = BeautifulSoup(html_content, "html.parser")

        for update_text_div in soup.find_all('div', {'class': 'updateText'}):
            h2_tag = update_text_div.find_previous('h2', {'class': ''})
            if h2_tag:
                title = h2_tag.get_text(strip=True)
                print(f"Title: {title}")

                div_tag = update_text_div.find_next('div', {'class': 'blogSysn'})
                if div_tag:
                    content = div_tag.get_text(strip=True)
                    
                    # Check if any sentiment keywords are present in the content
                    if any(keyword in content.lower() for keyword in sentiment_keywords):
                        sentiment_result = analyze_sentiment(content)
                        print(f"Paragraph Content: {content}")
                        print(f"Sentiment: {sentiment_result['label']} (confidence: {sentiment_result['score']:.4f})\n")

    except Exception as e:
        print(f"Error parsing the HTML: {e}")

def economictimes_headlines():
    print("Fetching headlines from Economic Times...")
    url = "https://economictimes.indiatimes.com/markets/stocks/live-blog/bse-sensex-today-live-nifty-stock-market-updates-02-january-2024/liveblog/msid-106462685,curpg-2.cms"

    try:
        page_request = requests.get(url)
        page_request.raise_for_status()
        print("Status:", page_request.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error making the request: {e}")
        return

    try:
        data = page_request.content
        sentiment_keywords = load_sentiment_keywords()
        read_html_data(data, sentiment_keywords)
    except Exception as e:
        print(f"Error parsing the page: {e}")

if __name__ == "__main__":
    economictimes_headlines()
